In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *
from fastai.column_data import *

In [ ]:
path='../input/'
tmp_path='/kaggle/working/tmp/'
models_path='/kaggle/working/models/'

1. read the files 

2. preprocess this data

    2.1. make a dict that maps from each user to its index, same with movies 
    
    2.2. get the number of users and movies

3.  split the data into x, y(inputs and targets) 

4. create your appropriate data object 

5. create a function that makes embeddings given the appropriate dimensions 

6. create the pytorch module or the model class 

7. set the loss function and optimizer and their hyperparameters 

8. TRAIN... 


In [ ]:
#First we read the ratings and movies csv
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

In [ ]:
movies = pd.read_csv(path+'movies.csv') 
movies.head()

In [ ]:
val_idxs = get_cv_idxs(len(ratings))
n_factors = 50 

In [ ]:
uniq_users = ratings.userId.unique()
user2idx = {i:o for o, i in enumerate(uniq_users)} 
ratings.userId = ratings.userId.apply(lambda x: user2idx[x]) 

uniq_movies = ratings.movieId.unique()
movie2idx = {i:o for o, i in enumerate(uniq_movies)} 
ratings.movieId = ratings.movieId.apply(lambda x: movie2idx[x]) 

n_movies = len(uniq_movies) 
n_users = len(uniq_users)

n_users, n_movies

In [ ]:
x = ratings.drop(['rating', 'timestamp'], axis = 1)
y = ratings['rating'].astype(np.float32)

In [ ]:
data = ColumnarModelData.from_data_frame(path, val_idxs, x, y, ['userId', 'movieId'], 64)

In [ ]:
min_rating,max_rating = ratings.rating.min(),ratings.rating.max()
min_rating,max_rating

In [ ]:
def get_emb(n,f):
    emb = nn.Embedding(n,f)
    emb.weight.data.uniform_(-0.01,0.01)
    return emb 

In [ ]:
class C_filter(nn.Module):
    def __init__(self, n_users, n_movies, nh = 10, p = [0.05, 0.5]):
        super().__init__()
        (self.u, self.m) = [get_emb(*o) for o in [(n_users, n_factors), (n_movies, n_factors)]]
        self.drop1 = nn.Dropout(p[0]) 
        self.drop2 = nn.Dropout(p[1]) 
        self.lin1 = nn.Linear(n_factors*2, nh)
        self.lin2 = nn.Linear(nh, 1) 
    def forward(self, cats, conts):
        users, movies = cats[:,0], cats[:,1] 
        x = self.drop1(torch.cat([self.u(users), self.m(movies)], dim = 1)) 
        x = self.drop2(F.relu(self.lin1(x)))
        return F.sigmoid(self.lin2(x)) * (max_rating - min_rating +1) + min_rating - 0.5

In [ ]:
wd = 1e-5
model = C_filter(n_users, n_movies).cuda() 
opt = optim.Adam(model.parameters(), 1e-3, weight_decay = wd)

In [ ]:
fit(model, data, 3, opt, F.mse_loss)

In [ ]:
set_lrs(opt, 1e-3)


In [ ]:
fit(model, data, 5, opt, F.mse_loss)
